#### Librerias

In [1]:
import pandas as pd
from datetime import datetime
import itertools
import numpy as np
import Proyecto as pro
import math

#### Funciones

In [2]:
def haversine(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferencias entre las coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radio de la Tierra en kilómetros (aproximadamente 6371)
    R = 6371
    distancia = R * c
    
    return distancia

def combine_datetime(date, time):
    try:
        # Intentar combinar fecha y hora
        return pd.to_datetime(date + ' ' + time)
    except Exception:
        # Si hay un error (por ejemplo, hora incompleta), asignar '00:00:00'
        return pd.to_datetime(date + ' ' + '00:00:00')

def millas_a_kilometros(millas):
    return millas * 1.60934

def calcular_distancia(hora_inicio, hora_fin, velocidad_promedio=80):
    # Convertir las horas a formato datetime
    hora_inicio = pd.to_datetime(hora_inicio)
    hora_fin = pd.to_datetime(hora_fin)
    
    # Calcular la duración en horas
    duracion_horas = (hora_fin - hora_inicio).dt.total_seconds() / 3600
    
    # Calcular la distancia recorrida (distancia = velocidad promedio * tiempo)
    distancia_recorrida = velocidad_promedio * duracion_horas
    
    return distancia_recorrida

def extraer_hms(hora_inicio, hora_fin):
    diferencias = []
    
    for inicio, fin in zip(hora_inicio, hora_fin):
        diferencia = fin - inicio
        
        # Extraer horas, minutos y segundos de la diferencia
        horas = diferencia.seconds // 3600
        minutos = (diferencia.seconds % 3600) // 60
        segundos = diferencia.seconds % 60
        
        # Agregar la diferencia formateada a la lista de resultados
        diferencias.append(f"{horas:02}:{minutos:02}:{segundos:02}")
    
    return diferencias

#### Lectura de datos

In [3]:
uber = pd.read_csv('DataSets/My Uber Drives - 2016.csv')
amazon = pd.read_csv('DataSets/amazon_delivery.csv')
dataset = pd.read_csv('DataSets/dataset.csv')

#### Limpieza y transformación

In [ ]:
#Para Horas
#Solo usar en la primera activacion del kernel o abra que reiniciarlo cada vez
uber['START_DATE*'] = pd.to_datetime(uber['START_DATE*'], format='%m/%d/%Y %H:%M').dt.strftime('%Y-%m-%d %H:%M:%S')
uber['END_DATE*'] = pd.to_datetime(uber['END_DATE*'], format='%m/%d/%Y %H:%M').dt.strftime('%Y-%m-%d %H:%M:%S')


temp_uber_SD = pd.to_datetime(uber['START_DATE*'])
temp_uber_ED = pd.to_datetime(uber['END_DATE*'])

temp_amazon_SD = amazon.apply(lambda row: combine_datetime(row['Order_Date'], row['Order_Time']), axis=1)
temp_amazon_ED = amazon.apply(lambda row: combine_datetime(row['Order_Date'], row['Pickup_Time']), axis=1)

temp_dataset_SD = pd.to_datetime(dataset['created_at'])
temp_dataset_ED = pd.to_datetime(dataset['actual_delivery_time'])

#En uber tengo millas
#en amazon tengo latitudes y regresa en kilometros

#Distancia
temp_uber_dist = millas_a_kilometros(uber['MILES*'])
temp_amazon_dist = haversine(amazon['Store_Latitude'], amazon['Store_Longitude'],amazon['Drop_Latitude'], amazon['Drop_Longitude'])
temp_dataset_dist = calcular_distancia(dataset['created_at'], dataset['actual_delivery_time'])

#Tiempo neto
temp_uber_time = extraer_hms(temp_uber_SD, temp_uber_ED)
temp_amazon_time = extraer_hms(temp_amazon_SD, temp_amazon_ED)
temp_dataset_time = extraer_hms(temp_dataset_SD, temp_dataset_ED)

#Categorias
temp_uber_cap = ["viaje"] * len(dataset)
temp_amazon_cap = ["e-commerce"] * len(amazon)
temp_dataset_cap = ["celiveri"] * len(dataset)

#Veiculo
temp_uber_veh = ["car"] * len(uber)
temp_amazon_veh = amazon['Vehicle']
temp_dataset_veh = ["scooter"] * len(dataset)



In [5]:
len(uber)

1156

#### Union

In [6]:
fecha_inicio = [temp_amazon_SD, temp_dataset_SD, temp_uber_SD]
fecha_fin = [temp_amazon_ED, temp_dataset_ED, temp_uber_ED]
tiempo = [temp_amazon_time, temp_dataset_time, temp_uber_time]
distancia = [temp_amazon_dist, temp_dataset_dist, temp_uber_dist]
categoria = [temp_amazon_cap, temp_dataset_cap, temp_uber_cap]
vehiculo = [temp_amazon_veh, temp_dataset_veh, temp_uber_veh]


fecha_inicio_flat = list(itertools.chain(*fecha_inicio))
fecha_fin_flat = list(itertools.chain(*fecha_fin))
tiempo_flat = list(itertools.chain(*tiempo))
distancia_flat = list(itertools.chain(*distancia))
categoria_flat = list(itertools.chain(*categoria))
vehiculo_flat = list(itertools.chain(*vehiculo))

dict_temp = {
    'Fecha Inicio': fecha_inicio_flat,
    'Fecha Fin': fecha_fin_flat,
    'Tiempo': tiempo_flat,
    'Distancia': distancia_flat,
    'Categoria': categoria_flat,
    'Vehiculo': vehiculo_flat
}

BasedeDatos = pd.DataFrame(dict_temp)
BasedeDatos = BasedeDatos.dropna(subset=['Fecha Inicio'])
BasedeDatos = BasedeDatos.dropna(subset=['Fecha Fin'])

In [7]:
BasedeDatos.head()

,Fecha Inicio,Fecha Fin,Tiempo,Distancia,Categoria,Vehiculo
0,2022-03-19 11:30:00,2022-03-19 11:45:00,00:15:00,3.025149,Clothing,motorcycle
1,2022-03-25 19:45:00,2022-03-25 19:50:00,00:05:00,20.183530,Electronics,scooter
2,2022-03-19 08:30:00,2022-03-19 08:45:00,00:15:00,1.552758,Sports,motorcycle
3,2022-04-05 18:00:00,2022-04-05 18:10:00,00:10:00,7.790401,Cosmetics,motorcycle
4,2022-03-26 13:30:00,2022-03-26 13:45:00,00:15:00,6.210138,Toys,scooter


In [8]:
Calculo = pro.MetodoDicotomico(BasedeDatos, 0.1, 0.5)
inter = Calculo.aplicarMetodo()
print(inter)

(Timedelta('0 days 09:19:22.447215271'), Timedelta('0 days 09:19:22.811437607'))
